In [1]:
import torch
from diffusers import AutoencoderKLWan
from diffusers.utils import export_to_video
from processor import WanAttnProcessor2_0
from pipeline import WanPipeline 
# Available models: Wan-AI/Wan2.1-T2V-14B-Diffusers, Wan-AI/Wan2.1-T2V-1.3B-Diffusers
model_id = "Wan-AI/Wan2.1-T2V-1.3B-Diffusers"
vae = AutoencoderKLWan.from_pretrained(model_id, subfolder="vae", torch_dtype=torch.float32)
pipe = WanPipeline.from_pretrained(model_id, vae=vae, torch_dtype=torch.bfloat16)
pipe.to("cuda")


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

WanPipeline {
  "_class_name": "WanPipeline",
  "_diffusers_version": "0.33.1",
  "_name_or_path": "Wan-AI/Wan2.1-T2V-1.3B-Diffusers",
  "scheduler": [
    "diffusers",
    "UniPCMultistepScheduler"
  ],
  "text_encoder": [
    "transformers",
    "UMT5EncoderModel"
  ],
  "tokenizer": [
    "transformers",
    "T5TokenizerFast"
  ],
  "transformer": [
    "diffusers",
    "WanTransformer3DModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKLWan"
  ]
}

In [29]:
prompt = "A cat on the grass"
neg_prompt = "low quality, blurry, bad lighting, weird motion"
pos_prompt_embeds, _ = pipe.encode_prompt(
    prompt=prompt,
    do_classifier_free_guidance=False
)

neg_prompt_embeds, _ = pipe.encode_prompt(
    prompt=neg_prompt,
    padding=False,
    do_classifier_free_guidance=False,
)
neg_len = neg_prompt_embeds.shape[1]
pos_len = pos_prompt_embeds.shape[1]

mask = None
# torch.zeros((1, 14040, pos_len+neg_len)).cuda()
# mask[:, :, -neg_len:] = -0.8

for block in pipe.transformer.blocks:
    block.attn2.processor = WanAttnProcessor2_0(scale=0.3, neg_prompt_length=neg_len, attn_mask=mask)

prompt_embeds = torch.cat([pos_prompt_embeds, neg_prompt_embeds], dim=1)

True
False


In [30]:
output = pipe(
    prompt_embeds=prompt_embeds,
    height=480,
    width=832,
    num_frames=81,
    num_inference_steps=50,
    guidance_scale=0.0,
    generator=torch.Generator(device="cuda").manual_seed(0),
).frames[0] 
export_to_video(output, "output.mp4", fps=15)

  0%|          | 0/50 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


'output.mp4'